<a href="https://colab.research.google.com/github/aarsanjani/meansquares/blob/master/version2/modeling/where_modeling/CA_KNN_MergeSupetSetData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Goal

This colab is for KNN similarity 
* to find the counties from Oregon, that are similar to CA widespread counties


# Mount drive

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="2"
import warnings; warnings.simplefilter('ignore')

In [ ]:
#drive.flush_and_unmount(600000)

In [3]:
!ls '/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021'

 All_CovidDataFeb20.csv
 alldatasets_Feb20.csv
 All_GoogleMobilityDataFeb20.csv
 All_SocialDistancingDataFeb20.csv
 AllStates_Superset_dataset_Feb20.csv
 AllStates_Superset_dataset_Feb21.csv
 CA_alldatasets_Feb05.csv
 CA_alldatasets_Jan13.csv
 CA_CombinedData_Labeled_CDCGuideline_Feb05.csv
 CA_CombinedData_Labeled_CDCGuideline_Feb06.csv
 CA_CombinedData_Labeled_CDCGuideline_Jan15.csv
 CA_combined_Feb04.csv
 CA_CovidDataJan10.csv
 CA_dataset_Feb11.csv
 CA_GoogleMobilityDataJan10.csv
 CA_MarylandData_Labeled_CDCGuideline_Feb05.csv
 CA_MarylandData_Labeled_CDCGuideline_Feb06.csv
 CA_MarylandData_Labeled_CDCGuideline_Jan15.csv
 CA_processedMarylandData_Feb04.csv
 CA_processedMarylandData_Feb05.csv
 CA_processedMarylandData_Feb12.csv
 CA_processedMarylandData_Jan11.csv
 CA_SocialDistancingDataJan10.csv
 CA_socialDistancingInertiaData_Feb04.csv
 CA_SuperData_Labeled_CDCGuideline_Feb13.csv
 CA_Superset_bins_dataset_Feb20.csv
 CA_Superset_dataset_Feb12.csv
 computed_CA_mobility_data.csv
 data-

# Import package

In [4]:
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler as ss
import itertools
from scipy import linalg
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz
import numpy as np
import matplotlib.pyplot as plt

# Load Oregon Binning Superset Data

In [77]:
or_bin_superset_data = pd.read_csv('/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/OR_Superset_bins_dataset_Feb21.csv',low_memory=False)



In [78]:
print(or_bin_superset_data.shape)


(364, 60)


In [79]:
or_bin_superset_data.head(2)

,Date,Province_State,FIPS,New cases,County Name,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity,State_Name,State,CTFIPS,CTNAME,STFIPS,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,New Cases/100k population,rolling_avg_new_cases/100k
0,2020-10-02,Oregon,41067,65,Washington County,10.0,21.121,107,-21.0,-7.0,0.0,-43.0,-43.0,13.0,310.164459,Oregon,OR,41067,Washington County,41,28,26,3.52,28.5,2.6,35.9,0.37,3.16,65,597695,6.2,6.13,18,78010,1.9,16.5,49.42,354,132,2.2,0.2,0.024,0.0736,1.68,0,50.99648,4.9,166.39,5.73,15,636,5.51,0,0.9,7.2,32.9,1.71,2.44,10.875112,6.286054
1,2020-10-02,Oregon,41067,65,Washington County,10.0,7.931,104,-21.0,-7.0,0.0,-43.0,-43.0,13.0,310.164459,Oregon,OR,41067,Washington County,41,28,26,3.52,28.5,2.6,35.9,0.37,3.16,65,597695,6.2,6.13,18,78010,1.9,16.5,49.42,354,132,2.2,0.2,0.024,0.0736,1.68,0,50.99648,4.9,166.39,5.73,15,636,5.51,0,0.9,7.2,32.9,1.71,2.44,10.875112,7.050897


#  Load CA Superset Labeled Data

In [80]:
ca_labeled_superset_data = pd.read_csv('/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/CA_SuperData_Labeled_CDCGuideline_Feb13.csv',low_memory=False)
ca_labeled_superset_data.head()

,Date,Province_State,FIPS,New cases,County Name,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity,State_Name,State,CTFIPS,CTNAME,STFIPS,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,New cases/100k,positivity_rate,rolling_avg_new_cases/100k,growth_label
0,2020-03-01,California,6001,0,Alameda County,0.0,3.684,58,6.0,9.0,15.0,1.0,4.0,0.0,860.757075,California,CA,6001,Alameda County,6,37,26,2.93,25.4,0.4,36.8,0.25,2.68,0,1666753,-5.1,15.04,19,92574,10.5,22.5,49.11,1155,131,2.05,0.22,0.0,0.0000,0.0,37,54.0,0.0,0.00,0.0,0,0,0.05,35,1.1,5.3,5.7,0.6,0.00,0.000000,0.000,0.0,minimal
1,2020-03-02,California,6001,0,Alameda County,0.0,6.182,98,4.0,7.0,7.0,-1.0,3.0,0.0,860.757075,California,CA,6001,Alameda County,6,17,20,3.38,28.2,0.4,38.1,0.63,2.74,0,1666753,-2.9,15.04,19,92574,10.5,22.5,49.11,1155,131,2.05,0.22,0.0,0.0000,0.0,38,54.0,0.0,0.00,0.0,0,0,0.05,35,1.1,5.3,5.7,0.6,0.00,0.000000,0.000,0.0,minimal
2,2020-03-03,California,6001,1,Alameda County,0.0,6.271,100,5.0,12.0,41.0,0.0,1.0,-1.0,860.757075,California,CA,6001,Alameda County,6,16,18,3.61,27.1,0.3,32.7,0.67,2.94,1,1666753,4.1,15.04,19,92574,10.5,22.5,49.11,1155,131,2.05,0.22,0.0,0.0002,0.0,0,54.0,0.0,0.00,0.0,0,1,0.05,42,1.1,5.3,5.7,0.6,0.00,0.059997,0.000,0.0,minimal
3,2020-03-04,California,6001,0,Alameda County,0.0,6.209,99,1.0,6.0,23.0,-3.0,1.0,0.0,860.757075,California,CA,6001,Alameda County,6,14,18,3.55,27.9,0.3,36.9,0.68,2.87,0,1666753,1.7,15.04,19,92574,10.5,22.5,49.11,1155,131,2.05,0.22,0.0,0.0002,0.0,0,54.0,10.3,0.01,0.0,0,1,0.06,42,1.1,5.3,5.7,0.6,0.04,0.000000,0.002,0.0,minimal
4,2020-03-05,California,6001,0,Alameda County,0.0,6.654,106,1.0,6.0,12.0,-5.0,-1.0,1.0,860.757075,California,CA,6001,Alameda County,6,15,19,3.54,28.1,0.3,36.8,0.66,2.88,0,1666753,2.0,15.04,19,92574,10.5,22.5,49.11,1155,131,2.05,0.22,0.0,0.0002,0.0,0,54.0,10.3,0.01,0.0,0,1,0.06,42,1.1,5.3,5.7,0.6,0.04,0.000000,0.002,0.0,minimal


In [81]:
ca_labeled_superset_data['State'].unique()

array(['CA'], dtype=object)

In [82]:
ca_labeled_superset_data.shape

(15391, 62)

In [83]:
ca_labeled_superset_data.growth_label.unique()

array(['minimal', 'moderate', 'substantial', 'widespread'], dtype=object)

In [84]:
#ca_labeled_superset_data = ca_labeled_superset_data[ca_labeled_superset_data['growth_label']=='widespread']

In [85]:
ca_labeled_superset_data.shape

(15391, 62)

In [86]:
print("CA labeled:", ca_labeled_superset_data.Date.min(),ca_labeled_superset_data.Date.max())
print("Oregon Bining:", or_bin_superset_data.Date.min(),or_bin_superset_data.Date.max())

CA labeled: 2020-03-01 2020-12-28
Oregon Bining: 2020-10-02 2020-10-15


# Column Mapping

In [87]:
ca_labeled_superset_data.columns

Index(['Date', 'Province_State', 'FIPS', 'New cases', 'County Name',
       'mask_rule_severity', 'm50', 'm50_index',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'PopDensity', 'State_Name',
       'State', 'CTFIPS', 'CTNAME', 'STFIPS', 'Social.distancing.index',
       'X..staying.home', 'Trips.person', 'X..out.of.county.trips',
       'X..out.of.state.trips', 'Miles.person', 'Work.trips.person',
       'Non.work.trips.person', 'New.COVID.cases', 'Population',
       'X..change.in.consumption', 'Transit.mode.share',
       'X..people.older.than.60', 'Median.income', 'X..African.Americans',
       'X..Hispanic.Americans', 'X..Male', 'Employment.density',
       'X..hot.spots.1000.people', 'Hospital.beds.1000.peop

In [88]:
or_bin_superset_data.columns

Index(['Date', 'Province_State', 'FIPS', 'New cases', 'County Name',
       'mask_rule_severity', 'm50', 'm50_index',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'PopDensity', 'State_Name',
       'State', 'CTFIPS', 'CTNAME', 'STFIPS', 'Social.distancing.index',
       'X..staying.home', 'Trips.person', 'X..out.of.county.trips',
       'X..out.of.state.trips', 'Miles.person', 'Work.trips.person',
       'Non.work.trips.person', 'New.COVID.cases', 'Population',
       'X..change.in.consumption', 'Transit.mode.share',
       'X..people.older.than.60', 'Median.income', 'X..African.Americans',
       'X..Hispanic.Americans', 'X..Male', 'Employment.density',
       'X..hot.spots.1000.people', 'Hospital.beds.1000.peop

In [89]:
required_cols = [ 'mask_rule_severity', 'm50_index', #'FIPS',
        'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'PopDensity',
        'Social.distancing.index','X..staying.home', 'Trips.person', 'X..out.of.county.trips',
       'X..out.of.state.trips', 'Miles.person', 'Work.trips.person',
       'Non.work.trips.person',#'Population',
       'X..change.in.consumption', 'Transit.mode.share',
       'X..people.older.than.60', 'Median.income', 'X..African.Americans',
       'X..Hispanic.Americans', 'X..Male','Employment.density',
       'X..hot.spots.1000.people', 'Hospital.beds.1000.people',
       'ICUs.1000.people','X.days..decreasing.COVID.cases', 'X..hospital.bed.utilization',
       'Testing.capacity', 'Tests.done.1000.people', 'X..ICU.utilization',
       'Ventilator.shortage', 'Imported.COVID.cases',
       'COVID.exposure.1000.people', 'X.days..decreasing.ILI.cases',
       'Unemployment.claims.1000.people', 'Unemployment.rate',
       'X..working.from.home', 'Cumulative.inflation.rate', 'COVID.death.rate',
       'rolling_avg_new_cases/100k']

In [90]:
ca_labeled_superset_data_filtered = ca_labeled_superset_data[required_cols]

In [91]:
or_bin_superset_data_filtered = or_bin_superset_data[required_cols]

# Use these columns

In [22]:
x_col_all_group = [ 'FIPS','mask_rule_severity', 'm50_index',
        'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'PopDensity',
        'Social.distancing.index','X..staying.home', 'Trips.person', 'X..out.of.county.trips',
       'X..out.of.state.trips', 'Miles.person', 'Work.trips.person',
       'Non.work.trips.person','Population',
       'X..change.in.consumption', 'Transit.mode.share',
       'X..people.older.than.60', 'Median.income', 'X..African.Americans',
       'X..Hispanic.Americans', 'X..Male','Employment.density',
       'X..hot.spots.1000.people', 'Hospital.beds.1000.people',
       'ICUs.1000.people','X.days..decreasing.COVID.cases', 'X..hospital.bed.utilization',
       'Testing.capacity', 'Tests.done.1000.people', 'X..ICU.utilization',
       'Ventilator.shortage', 'Imported.COVID.cases',
       'COVID.exposure.1000.people', 'X.days..decreasing.ILI.cases',
       'Unemployment.claims.1000.people', 'Unemployment.rate',
       'X..working.from.home', 'Cumulative.inflation.rate', 'COVID.death.rate',
       'rolling_avg_new_cases/100k']


# Remove FIPS ,

In [23]:
len(x_col_all_group)

45

# Option1: KNN



In [92]:
#https://towardsdatascience.com/cosine-similarity-how-does-it-measure-the-similarity-maths-behind-and-usage-in-python-50ad30aad7db
#### Reference Code 
# using sklearn to calculate cosine similarity
#from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
#cos_sim=cosine_similarity(A.reshape(1,-1),B.reshape(1,-1))
#print (f"Cosine Similarity between A and B:{cos_sim}")- high is good
#print (f"Cosine Distance between A and B:{1-cos_sim}")- distance less is good

In [93]:
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances

def key_consine_similarity(tupple):
    return tupple[1]

# This method computes the distance between each vectors with predicted_vectors input
# in our case, vectors is CA data , predicted_vectors is OR data

def get_computed_similarities(vectors, predicted_vectors, reverse=False):
    data_size = len(vectors) # size of your CA data
    print("len of OR:", len(predicted_vectors)) #size of OR
    print("len of CA:", data_size)
    cosine_similarities = []

    for i in range(len(predicted_vectors)): #OR
      max_cosine = -1
      max_j = 0
      for j in range(data_size): #CA
        cosine_sim_val = cosine_similarity(vectors[j].reshape(1,-1), predicted_vectors[i].reshape(1,-1)) #(1 - cosine(vectors[j], predicted_vectors[i]))
        if max_cosine < cosine_sim_val:
            max_cosine = cosine_sim_val
            max_j = j
      cosine_similarities.append((max_j, max_cosine))
        #if cosine_sim_val > 0.5: # update threshold
          #continue
        
    print('len cosine_similarities : ',len(cosine_similarities))
    #return sorted(cosine_similarities, key=key_consine_similarity, reverse=reverse)
    return cosine_similarities

def display_top_n(sorted_cosine_similarities, n=10):
    for i in range(n):
        index, consine_sim_val = sorted_cosine_similarities[i]
        print('News Title: ', merged_data.iloc[index, 0])  
        print('News statement: ', merged_data.iloc[index, 1])  
        print('News label: ', merged_data.iloc[index, 4])  

        print('Cosine Sim Val :', consine_sim_val)
        print('---------------------------------')

In [94]:
or_bin_superset_data_filtered.head(3)

,mask_rule_severity,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,X..change.in.consumption,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,rolling_avg_new_cases/100k
0,10.0,107,-21.0,-7.0,0.0,-43.0,-43.0,13.0,310.164459,28,26,3.52,28.5,2.6,35.9,0.37,3.16,6.2,6.13,18,78010,1.9,16.5,49.42,354,132,2.2,0.2,0,50.99648,4.9,166.39,5.73,15,636,5.51,0,0.9,7.2,32.9,1.71,2.44,6.286054
1,10.0,104,-21.0,-7.0,0.0,-43.0,-43.0,13.0,310.164459,28,26,3.52,28.5,2.6,35.9,0.37,3.16,6.2,6.13,18,78010,1.9,16.5,49.42,354,132,2.2,0.2,0,50.99648,4.9,166.39,5.73,15,636,5.51,0,0.9,7.2,32.9,1.71,2.44,7.050897
2,10.0,128,-21.0,-7.0,0.0,-43.0,-43.0,13.0,310.164459,28,26,3.52,28.5,2.6,35.9,0.37,3.16,6.2,6.13,18,78010,1.9,16.5,49.42,354,132,2.2,0.2,0,50.99648,4.9,166.39,5.73,15,636,5.51,0,0.9,7.2,32.9,1.71,2.44,7.815740


In [95]:
print(or_bin_superset_data_filtered.shape)
or_bin_superset_data.shape

(364, 43)


(364, 60)

In [96]:
or_bin_superset_data['similiary_score'] = get_computed_similarities(vectors=ca_labeled_superset_data_filtered.values, predicted_vectors=or_bin_superset_data_filtered.values)

len of OR: 364
len of CA: 15391
len cosine_similarities :  364


In [97]:
or_bin_superset_data.head(4)

,Date,Province_State,FIPS,New cases,County Name,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity,State_Name,State,CTFIPS,CTNAME,STFIPS,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,New Cases/100k population,rolling_avg_new_cases/100k,similiary_score
0,2020-10-02,Oregon,41067,65,Washington County,10.0,21.121,107,-21.0,-7.0,0.0,-43.0,-43.0,13.0,310.164459,Oregon,OR,41067,Washington County,41,28,26,3.52,28.5,2.6,35.9,0.37,3.16,65,597695,6.2,6.13,18,78010,1.9,16.5,49.42,354,132,2.2,0.2,0.024,0.0736,1.68,0,50.99648,4.9,166.39,5.73,15,636,5.51,0,0.9,7.2,32.9,1.71,2.44,10.875112,6.286054,"(10952, [[0.9999907527813767]])"
1,2020-10-02,Oregon,41067,65,Washington County,10.0,7.931,104,-21.0,-7.0,0.0,-43.0,-43.0,13.0,310.164459,Oregon,OR,41067,Washington County,41,28,26,3.52,28.5,2.6,35.9,0.37,3.16,65,597695,6.2,6.13,18,78010,1.9,16.5,49.42,354,132,2.2,0.2,0.024,0.0736,1.68,0,50.99648,4.9,166.39,5.73,15,636,5.51,0,0.9,7.2,32.9,1.71,2.44,10.875112,7.050897,"(10952, [[0.9999907971148785]])"
2,2020-10-02,Oregon,41067,65,Washington County,10.0,22.297,128,-21.0,-7.0,0.0,-43.0,-43.0,13.0,310.164459,Oregon,OR,41067,Washington County,41,28,26,3.52,28.5,2.6,35.9,0.37,3.16,65,597695,6.2,6.13,18,78010,1.9,16.5,49.42,354,132,2.2,0.2,0.024,0.0736,1.68,0,50.99648,4.9,166.39,5.73,15,636,5.51,0,0.9,7.2,32.9,1.71,2.44,10.875112,7.815740,"(10952, [[0.999990398846052]])"
3,2020-10-02,Oregon,41067,65,Washington County,10.0,15.267,192,-21.0,-7.0,0.0,-43.0,-43.0,13.0,310.164459,Oregon,OR,41067,Washington County,41,28,26,3.52,28.5,2.6,35.9,0.37,3.16,65,597695,6.2,6.13,18,78010,1.9,16.5,49.42,354,132,2.2,0.2,0.024,0.0736,1.68,0,50.99648,4.9,166.39,5.73,15,636,5.51,0,0.9,7.2,32.9,1.71,2.44,10.875112,8.580583,"(10952, [[0.9999888745768173]])"


In [98]:
or_bin_superset_data.similiary_score

0      (10952, [[0.9999907527813767]])
1      (10952, [[0.9999907971148785]])
2       (10952, [[0.999990398846052]])
3      (10952, [[0.9999888745768173]])
4       (10952, [[0.999986065018338]])
                    ...               
359     (10952, [[0.999991298827008]])
360    (10952, [[0.9999914074148615]])
361    (10952, [[0.9999916733050149]])
362    (10952, [[0.9999915877190941]])
363    (10952, [[0.9999918650275855]])
Name: similiary_score, Length: 364, dtype: object

In [99]:
or_bin_superset_data_filtered.head(2)

,mask_rule_severity,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,X..change.in.consumption,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,rolling_avg_new_cases/100k
0,10.0,107,-21.0,-7.0,0.0,-43.0,-43.0,13.0,310.164459,28,26,3.52,28.5,2.6,35.9,0.37,3.16,6.2,6.13,18,78010,1.9,16.5,49.42,354,132,2.2,0.2,0,50.99648,4.9,166.39,5.73,15,636,5.51,0,0.9,7.2,32.9,1.71,2.44,6.286054
1,10.0,104,-21.0,-7.0,0.0,-43.0,-43.0,13.0,310.164459,28,26,3.52,28.5,2.6,35.9,0.37,3.16,6.2,6.13,18,78010,1.9,16.5,49.42,354,132,2.2,0.2,0,50.99648,4.9,166.39,5.73,15,636,5.51,0,0.9,7.2,32.9,1.71,2.44,7.050897


## Save Temp File to CSV [For Analysis]

In [100]:
from datetime import datetime
location = '/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/'
filename = 'OR_SuperData_With_SimilarityScore_'+datetime.now().strftime("%b%d")+'.csv' #%Y%m%d
print(filename)
or_bin_superset_data.to_csv(location+filename,index=False)

OR_SuperData_With_SimilarityScore_Feb24.csv


## Getting growth_label in OR from CA

In [101]:
print(len(or_bin_superset_data))
#select the first row 
print(or_bin_superset_data.iloc[0]['similiary_score'])

print("CA row number=",or_bin_superset_data.iloc[0]['similiary_score'][0])

print(or_bin_superset_data.iloc[0]['similiary_score'][1])

364
(10952, array([[0.99999075]]))
CA row number= 10952
[[0.99999075]]


In [102]:
ca_labeled_superset_data.iloc[10952]['growth_label']

'minimal'

In [103]:
for i in range(0,len(or_bin_superset_data)):
  row = or_bin_superset_data.iloc[i]['similiary_score'][0]
  #print(row)
  or_bin_superset_data['growth_label'] = ca_labeled_superset_data.iloc[row]['growth_label']

In [104]:
or_bin_superset_data.head(2)

,Date,Province_State,FIPS,New cases,County Name,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity,State_Name,State,CTFIPS,CTNAME,STFIPS,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,New Cases/100k population,rolling_avg_new_cases/100k,similiary_score,growth_label
0,2020-10-02,Oregon,41067,65,Washington County,10.0,21.121,107,-21.0,-7.0,0.0,-43.0,-43.0,13.0,310.164459,Oregon,OR,41067,Washington County,41,28,26,3.52,28.5,2.6,35.9,0.37,3.16,65,597695,6.2,6.13,18,78010,1.9,16.5,49.42,354,132,2.2,0.2,0.024,0.0736,1.68,0,50.99648,4.9,166.39,5.73,15,636,5.51,0,0.9,7.2,32.9,1.71,2.44,10.875112,6.286054,"(10952, [[0.9999907527813767]])",minimal
1,2020-10-02,Oregon,41067,65,Washington County,10.0,7.931,104,-21.0,-7.0,0.0,-43.0,-43.0,13.0,310.164459,Oregon,OR,41067,Washington County,41,28,26,3.52,28.5,2.6,35.9,0.37,3.16,65,597695,6.2,6.13,18,78010,1.9,16.5,49.42,354,132,2.2,0.2,0.024,0.0736,1.68,0,50.99648,4.9,166.39,5.73,15,636,5.51,0,0.9,7.2,32.9,1.71,2.44,10.875112,7.050897,"(10952, [[0.9999907971148785]])",minimal


In [105]:
or_bin_superset_data['growth_label'].unique()

array(['minimal'], dtype=object)

In [106]:
or_bin_superset_data[or_bin_superset_data['growth_label']=='widespread']

,Date,Province_State,FIPS,New cases,County Name,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity,State_Name,State,CTFIPS,CTNAME,STFIPS,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,New Cases/100k population,rolling_avg_new_cases/100k,similiary_score,growth_label


# Option2:  KNN- Opposite- Add with CA- Not using now

https://towardsdatascience.com/knn-in-python-835643e2fb53

In [ ]:
ca_labeled_superset_data = ca_labeled_superset_data[ca_labeled_superset_data['growth_label']=='widespread']

In [ ]:
ca_labeled_superset_data_filtered = ca_labeled_superset_data[required_cols]

In [ ]:
or_bin_superset_data_filtered = or_bin_superset_data[required_cols]

In [ ]:
from scipy.spatial.distance import cosine

def key_consine_similarity(tupple):
    return tupple[1]

# This method computes the distance between each vectors with predicted_vectors input
# in our case, vectors is CA data , predicted_vectors is OR data

def get_computed_similarities_rev(vectors, predicted_vectors, reverse=False):
    data_size = len(vectors) # size of your CA data
    print("len of OR:", len(predicted_vectors)) #size of OR
    print("len of CA:", data_size)
    cosine_similarities = []

    for i in range(data_size): #CA
      max_cosine = -1
      max_j = 0
      for j in range(len(predicted_vectors)): #OR
        cosine_sim_val = (1 - cosine(vectors[i], predicted_vectors[j]))
        if max_cosine < cosine_sim_val:
            max_cosine = cosine_sim_val
            max_j = j
      cosine_similarities.append((max_j, max_cosine))
        #if cosine_sim_val > 0.5: # update threshold
          #continue
        
    print('len cosine_similarities : ',len(cosine_similarities))
    #return sorted(cosine_similarities, key=key_consine_similarity, reverse=reverse)
    return cosine_similarities

def display_top_n(sorted_cosine_similarities, n=10):
    for i in range(n):
        index, consine_sim_val = sorted_cosine_similarities[i]
        print('News Title: ', merged_data.iloc[index, 0])  
        print('News statement: ', merged_data.iloc[index, 1])  
        print('News label: ', merged_data.iloc[index, 4])  

        print('Cosine Sim Val :', consine_sim_val)
        print('---------------------------------')

In [ ]:
or_bin_superset_data_filtered.head(3)

,mask_rule_severity,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,X..change.in.consumption,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,rolling_avg_new_cases/100k
0,10.0,107,-21.0,-7.0,0.0,-43.0,-43.0,13.0,310.164459,28,26,3.52,28.5,2.6,35.9,0.37,3.16,6.2,6.13,18,78010,1.9,16.5,49.42,354,132,2.2,0.2,0,50.99648,4.9,166.39,5.73,15,636,5.51,0,0.9,7.2,32.9,1.71,2.44,6.286054
1,10.0,104,-21.0,-7.0,0.0,-43.0,-43.0,13.0,310.164459,28,26,3.52,28.5,2.6,35.9,0.37,3.16,6.2,6.13,18,78010,1.9,16.5,49.42,354,132,2.2,0.2,0,50.99648,4.9,166.39,5.73,15,636,5.51,0,0.9,7.2,32.9,1.71,2.44,7.050897
2,10.0,128,-21.0,-7.0,0.0,-43.0,-43.0,13.0,310.164459,28,26,3.52,28.5,2.6,35.9,0.37,3.16,6.2,6.13,18,78010,1.9,16.5,49.42,354,132,2.2,0.2,0,50.99648,4.9,166.39,5.73,15,636,5.51,0,0.9,7.2,32.9,1.71,2.44,7.815740


In [ ]:
print(or_bin_superset_data_filtered.shape)
print(ca_labeled_superset_data_filtered.shape)
or_bin_superset_data.shape

(364, 43)
(521, 43)


(364, 61)

In [ ]:
ca_labeled_superset_data_filtered['similiary_score_with_OR'] = get_computed_similarities_rev(vectors=ca_labeled_superset_data_filtered.values, predicted_vectors=or_bin_superset_data_filtered.values)

len of OR: 364
len of CA: 521
len cosine_similarities :  521


In [ ]:
ca_labeled_superset_data_filtered.head(4)

,mask_rule_severity,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,X..change.in.consumption,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,rolling_avg_new_cases/100k,similiary_score_with_OR
540,10.0,38,-6.0,-6.0,0.0,0.0,-22.0,8.0,24.565345,18,19,3.32,26.6,0.5,46.2,0.38,2.94,13.8,0.16,35,61198,2.0,13.6,53.67,20,147,2.05,0.22,0,65.511702,5.1,614.28,24.33,1104,202,23.08,259,3.3,10.6,38.1,1.71,2.13,71.096666,"(103, 0.9997803339022608)"
541,10.0,39,-7.0,-5.0,0.0,0.0,-23.0,8.0,24.565345,16,19,3.26,29.4,0.4,53.0,0.38,2.87,11.1,0.16,35,61198,2.0,13.6,53.67,20,147,2.05,0.22,0,66.029105,5.2,618.72,24.62,1163,233,24.74,259,3.3,10.6,38.1,1.71,2.00,92.860952,"(103, 0.9997628382121437)"
542,10.0,47,-9.0,-3.0,0.0,0.0,-22.0,8.0,24.565345,17,19,3.57,27.1,0.3,59.3,0.36,3.21,24.3,0.16,35,61198,2.0,13.6,53.67,20,147,2.05,0.22,0,66.583542,5.2,623.79,25.78,1222,270,26.08,259,3.3,10.6,38.1,1.71,1.91,113.174285,"(84, 0.9997440704629695)"
543,10.0,17,-5.0,0.0,0.0,0.0,-9.0,0.0,24.565345,26,24,2.99,39.2,1.1,58.8,0.24,2.75,6.5,0.16,35,61198,2.0,13.6,53.67,20,147,2.05,0.22,0,67.161747,5.3,629.52,25.98,1289,387,27.18,259,3.3,10.6,38.1,1.71,1.85,131.673927,"(243, 0.9997224799864756)"


In [ ]:
ca_labeled_superset_data_filtered.similiary_score_with_OR.unique()

array([(103, 0.9997803339022608), (103, 0.9997628382121437),
       (84, 0.9997440704629695), (243, 0.9997224799864756),
       (105, 0.999698764956809), (99, 0.9996752759502611),
       (89, 0.9996497500028938), (99, 0.9996256497014715),
       (105, 0.9995999284165753), (352, 0.9993017844854767),
       (240, 0.999261494109279), (352, 0.9992184815893761),
       (355, 0.9991713656367462), (352, 0.9991224079533975),
       (352, 0.9990681281400025), (352, 0.9990098206974501),
       (352, 0.9989479544034175), (352, 0.9988796444831441),
       (352, 0.9988093511629081), (87, 0.9999007625899466),
       (85, 0.9998986539457577), (87, 0.9998969439632971),
       (105, 0.9996486717963383), (96, 0.9995656033365777),
       (95, 0.9995356555653538), (243, 0.9991870096511825),
       (240, 0.9991398429340427), (242, 0.9990905766219971),
       (337, 0.999035983393285), (352, 0.998976702436995),
       (240, 0.9989142253977616), (268, 0.9988446002239827),
       (352, 0.9987757113144994), (35

In [ ]:
or_bin_superset_data.filter(like = '103', axis=0)

,Date,Province_State,FIPS,New cases,County Name,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity,State_Name,State,CTFIPS,CTNAME,STFIPS,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,New Cases/100k population,rolling_avg_new_cases/100k,similiary_score
103,2020-10-05,Oregon,41067,31,Washington County,10.0,2.926,50,-18.0,-10.0,22.0,-49.0,-44.0,14.0,310.164459,Oregon,OR,41067,Washington County,41,43,31,3.22,25.9,2.0,26.5,0.41,2.81,31,597695,-5.6,6.13,18,78010,1.9,16.5,49.42,354,132,2.2,0.2,0.024,0.0541,1.7,0,51.085148,4.9,169.19,5.13,15,470,5.69,0,1.0,6.9,32.6,1.71,2.4,5.186592,5.186592,"(10952, 0.9999878090813492)"


In [ ]:
or_bin_superset_data.filter(like = '84', axis=0)

,Date,Province_State,FIPS,New cases,County Name,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity,State_Name,State,CTFIPS,CTNAME,STFIPS,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,New Cases/100k population,rolling_avg_new_cases/100k,similiary_score
84,2020-10-05,Oregon,41067,31,Washington County,10.0,12.828,65,-18.0,-10.0,22.0,-49.0,-44.0,14.0,310.164459,Oregon,OR,41067,Washington County,41,43,31,3.22,25.9,2.0,26.5,0.41,2.81,31,597695,-5.6,6.13,18,78010,1.9,16.5,49.42,354,132,2.2,0.2,0.024,0.0541,1.7,0,51.085148,4.9,169.19,5.13,15,470,5.69,0,1.0,6.9,32.6,1.71,2.40,5.186592,5.473408,"(10952, 0.9999877045172133)"
184,2020-10-09,Oregon,41067,35,Washington County,10.0,4.027,27,-20.0,-8.0,0.0,-47.0,-44.0,13.0,310.164459,Oregon,OR,41067,Washington County,41,46,26,3.06,30.4,3.2,33.7,0.37,2.69,35,597695,-9.6,6.13,18,78010,1.9,16.5,49.42,354,132,2.2,0.2,0.024,0.0647,1.9,1,51.338100,5.0,174.35,5.85,16,670,5.92,7,1.0,6.9,32.6,1.71,2.41,5.855829,5.855829,"(10952, 0.9999919192925663)"
284,2020-10-13,Oregon,41067,46,Washington County,10.0,4.210,179,-24.0,-8.0,-4.0,-54.0,-45.0,16.0,310.164459,Oregon,OR,41067,Washington County,41,33,27,3.45,27.2,2.6,30.4,0.41,3.04,46,597695,2.2,6.13,18,78010,1.9,16.5,49.42,354,132,2.2,0.2,0.024,0.0792,2.0,0,51.368106,5.0,179.27,5.61,16,650,6.10,14,1.1,6.7,32.5,1.71,2.37,7.696233,7.122601,"(10952, 0.9999898742320167)"


In [ ]:
or_bin_superset_data.filter(like = '243', axis=0)

,Date,Province_State,FIPS,New cases,County Name,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity,State_Name,State,CTFIPS,CTNAME,STFIPS,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,New Cases/100k population,rolling_avg_new_cases/100k,similiary_score
243,2020-10-11,Oregon,41067,62,Washington County,10.0,1.412,24,-24.0,-14.0,-45.0,-50.0,-11.0,6.0,310.164459,Oregon,OR,41067,Washington County,41,41,33,3.42,24.6,2.4,38.7,0.19,3.22,62,597695,8.2,6.13,18,78010,1.9,16.5,49.42,354,132,2.2,0.2,0.024,0.0809,1.99,0,51.406607,5.0,176.88,5.85,17,614,6.03,7,1.1,6.7,32.5,1.71,2.37,10.373184,10.373184,"(10952, 0.9999914069356016)"


In [ ]:
or_bin_superset_data.filter(like = '105', axis=0)

,Date,Province_State,FIPS,New cases,County Name,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity,State_Name,State,CTFIPS,CTNAME,STFIPS,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,New Cases/100k population,rolling_avg_new_cases/100k,similiary_score
105,2020-10-05,Oregon,41067,31,Washington County,10.0,3.963,37,-18.0,-10.0,22.0,-49.0,-44.0,14.0,310.164459,Oregon,OR,41067,Washington County,41,43,31,3.22,25.9,2.0,26.5,0.41,2.81,31,597695,-5.6,6.13,18,78010,1.9,16.5,49.42,354,132,2.2,0.2,0.024,0.0541,1.7,0,51.085148,4.9,169.19,5.13,15,470,5.69,0,1.0,6.9,32.6,1.71,2.4,5.186592,5.186592,"(10952, 0.999987869778439)"


In [ ]:
or_bin_superset_data.filter(like = '324', axis=0)

,Date,Province_State,FIPS,New cases,County Name,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity,State_Name,State,CTFIPS,CTNAME,STFIPS,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,New Cases/100k population,rolling_avg_new_cases/100k,similiary_score
324,2020-10-14,Oregon,41067,50,Washington County,10.0,6.61,44,-18.0,-9.0,31.0,-52.0,-43.0,15.0,310.164459,Oregon,OR,41067,Washington County,41,41,28,2.7,32.3,3.5,32.0,0.41,2.29,50,597695,-23.0,6.13,18,78010,1.9,16.5,49.42,354,132,2.2,0.2,0.024,0.0725,2.09,0,51.327364,5.0,181.72,5.97,16,713,6.15,14,1.1,6.7,32.5,1.71,2.36,8.365471,8.365471,"(10952, 0.9999919255715619)"


In [ ]:
#sorted_cosine_similarities = get_computed_similarities(vectors=CA_data.values, predicted_vectors=OR_data.values)
#display_top_n(sorted_cosine_similarities=sorted_cosine_similarities)

In [ ]:
or_bin_superset_data_filtered

# Conclusion, Details, and Observation

* I have fixed the and changed the logic of cosine similarity code
* There are two options, I have followed to compute the similarity score for Oregon

> 1. **Option1:** A= Oregon , B= CA ; Outer loop= OR, Inner Loop= CA
>> * I have taken Oregon data and all the labeled CA data
>> * Then I have compared each row of Oregon with all the rows of CA labeled data. I cann't filter here the 'superspread' CA labeled data, because then OR data will be forcefully tagged as 'superspread'.
>> * I have stored the similarity score and the CA dataframe's row number pair in Oregon dataframe. 
>> * So now each row of Oregon dataset has the similarity score with the matching row number of CA dataframe.

> **2. Option2:** A= CA-Superspread B= Oregon ,  ; Outer loop= CA-Superspread, Inner Loop= OR
>> * This is opposite of option-1 and I think this the is the best way.
>> * Here I have filtered the 'superspread' data of CA labeled data
>> * Then I have compared the 'superspread' CA data with each rows of Oregon data to compute the cosine similarity
>> * Final output, for each row in 'superspread' CA data, I am storing the matching/ high similarity row number from Oregon dataset.
>> * Here we have to analize less number of data and only the superspread data
>> * I have analized few rows and found 'Washington County' of Oregon as a superspread county and high similarity with CA superspread data. This is matching with current situation also.

# Our old code for KNN Classifier

In [ ]:
# KNN classifier

seed = 58

# perform the split which gets us the train data and the test data
X_train, y_train, X_test, y_test = train_test_split(CA_superset_data_filtered, labels,
                                                                    test_size=0.25,
                                                                    random_state=seed,
                                                                    stratify=labels)
knn = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='cosine',
                           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
                           weights='distance')

knn.fit(X_train, y_train)
labels_pred = knn.predict(X_test)


## Jeyasri's code ( for Reference)

In [ ]:
from scipy.spatial.distance import cosine

def key_consine_similarity(tupple):
    return tupple[1]

# This method computes the distance between each vectors with predicted_vectors input
# in our case, vectors is CA data , predicted_vectors is OR data

def get_computed_similarities(vectors, predicted_vectors, reverse=False):
    data_size = len(vectors) # size of your CA data
    cosine_similarities = []

    for i in range(len(predicted_vectors)):
      for j in range(data):
        cosine_sim_val = (1 - cosine(vectors[j], predicted_vectors[i]))
        cosine_similarities.append((i, cosine_sim_val))
        if cosine_sim_val > 0.5: # update threshold
          continue
        
    print('len cosine_similarities : ',len(cosine_similarities))
    return sorted(cosine_similarities, key=key_consine_similarity, reverse=reverse)

def display_top_n(sorted_cosine_similarities, n=10):
    for i in range(n):
        index, consine_sim_val = sorted_cosine_similarities[i]
        print('News Title: ', merged_data.iloc[index, 0])  
        print('News statement: ', merged_data.iloc[index, 1])  
        print('News label: ', merged_data.iloc[index, 4])  

        print('Cosine Sim Val :', consine_sim_val)
        print('---------------------------------')

In [ ]:
df['similiary_score'] = get_computed_similarities()

In [ ]:
sorted_cosine_similarities = get_computed_similarities(vectors=CA_data.values, predicted_vectors=OR_data.values)
display_top_n(sorted_cosine_similarities=sorted_cosine_similarities)